# Modeling an orogenic system using FastScape: 1. the basic equations

![Henry Mountains](HenryMountains.jpg "Henry Mountains")

The source area is the mountain. This is where erosion takes place (mostly).

Two processes compete in the mountain: bedrock incision along channels and hillslope transport (or diffusion). It is commonly assumed that bedrock incision sets the pace of landscape evolution. Hillslope processes react to slope formation and amplification produced by channel incision. There are many potential feedbacks to this, including the tools provided by the hilslope processes or the perturbation to river flow by the product of hillslope processes (landslides).

We will neglect the effect of glacial processes.

## Basic equation

The basic equation governing landscape evolution (through erosion) in a mountain or orogenic area is:

$$\frac{\partial h}{\partial t}=U-K_fA^mS^n+K_d\nabla^2h$$

where $h$ is topographic elevation or height (in m), $t$ is time (in yrs), $U$ is uplift rate (in m/yr), $A$ is drainage area in m$^2$, $S$ is slope (in m/m), and $\nabla^2 h$ is curvature (second order derivative of topography, in m/m$^2$). $K_f$ is the bedrock incision rate coefficient (in m$^{1-2m}$/yr) that depends on lithology (i.e., rock type, fracturing level, etc) and precipitation rate, $\nu$, to the power $m$. $K_d$ is hillslope transport coeeficient (or diffusivity, in m$^2$/yr). $m$ and $n$ are the area and slope exponents that have values varying between $m\in[0.3,0.8]$ and $n\in[1-2]$; the ratio $m/n$ is better constrained by the concavity of river profiles to be $\in[0.4-0.6]$.

## FastScape algorithm

Solving this equation is not easy, especially because of the river incision term (or Stream Power Law - SPL term):

$$\frac{\partial h}{\partial t}=-K_fA^mS^n=-K_fA^m(\frac{\partial h}{\partial s})^n$$

because the slope, $S$, is in fact the spatial derivative of the topography, i.e., in the direction $s$ of water flow, in this case the steepest descent. This leads to an equation in the form of an *advection* equation, which is known to be prone to instability and inaccuracy.

The FastScape algorithm, developed by [Braun and Willett, 2013](https://www.sciencedirect.com/science/article/pii/S0169555X12004618), is unconditionally stable, highly efficient and relatively accurate, although it suffers from problems, i.e., it is not weel suited to study the propagation of knockpoints, which it tends to smooth out (see [Campforts et al, 2015](https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1002/2014JF003376)).

It is based on an ordering of the nodes that is obtained after defining the receiver node to each node, from which the donor nodes to each node are computer.

![Donor-Receiver](DonorReceiver.jpg "Donor-Receiver")

## Solution of the diffusion equation/term

There are many different ways to solve the diffusion equaiont. If the diffusivity, $K_d$, is spatially uniform, an efficient and unconditionally stable algorithm exists: the Alternating Direction Implicit (or ADI) algorithm (see [ADI](https://en.wikipedia.org/wiki/Alternating_direction_implicit_method)).

## FastScape Python implementation

Over the past few years, we have developed a Python package to solve the SPL and diffusion equations. It can be foound here: [FastScape on GitHub](https://github.com/fastscape-lem/fastscape). Documentation can be found here: [FastScape Docs](https://fastscape.readthedocs.io/en/latest/index.html).

FastScape was built using the *Xarray-Simlab* framework, developed at the GFZ by Benoit Bovy. Documentation can be found here: [Xarray-Simlab](https://xarray-simlab.readthedocs.io/en/latest/).

## Setup

To solve a Partial Differential Equation (PDE) as the one shown above, one also needs to specify the geometry of the problem and, most importantly, boundary conditions. In FastScape, the PDE is solved in a rectangular domain of dimensions $L_x\times L_y$ which is discretized using a rectangular (regular) discretization of $n_x\times n_y$ points or *nodes*.

Along the four boundaries, boundary conditions can be of three types:
1. *fixed_value*: the elevation is fixed to base level; note that rivers must "exit" through a base level, which implies that at least one of the boundaries MUST be fixed;
2. *core*: the boundary is reflective (no flux of material allowed), which means that rivers must "bounce back" from such boundaries;
3. *looped*: or cyclic boundary conditions, which means that material flowing through the boundary "reappears" automatically on the opposite boundary; note that *looped* boundaries must be specified as opposite pairs.

![DMesh](Mesh.jpg "Mesh")

FInally, FastScape also needs an initial condition, i.e.:

$$h(t=0)=h_0(x,y)$$

which is commonly assumed to be a very small white noise (i.e. of amplitude = 1 m).